In [2]:
# Import packages
import pandas as pd
import numpy as np
import yfinance as yf
import pandas_datareader as pdr
import statsmodels.api as sm
import math as ma

In [3]:
def indent():
    print()
    print()

In [4]:
# Load Fama-French factor data
ff3f = pdr.DataReader('F-F_Research_Data_Factors', 'famafrench', '2018-12-31', '2024-12-31')[0]/100
ff3f.head(5)
ff3f.iloc[:,:]
#Gives you most recent RF rate too (found at bottom of printed list)

,Mkt-RF,SMB,HML,RF
Date,,,,
2018-12,-0.0957,-0.0238,-0.0184,0.0020
2019-01,0.0840,0.0290,-0.0046,0.0021
2019-02,0.0340,0.0205,-0.0267,0.0018
2019-03,0.0110,-0.0305,-0.0417,0.0019
2019-04,0.0397,-0.0174,0.0215,0.0021
...,...,...,...,...
2023-10,-0.0319,-0.0387,0.0019,0.0047
2023-11,0.0884,-0.0002,0.0164,0.0044
2023-12,0.0487,0.0634,0.0493,0.0043


In [5]:
    # Download monthly prices (keep only Adjusted Close prices)
ticker = 'MSFT'   
    
firm_prices = yf.download(ticker, '2018-12-31', '2024-01-31', interval = '1mo')['Adj Close'].dropna().to_frame()

    # Calculate monthly returns, drop missing, convert from Series to DataFrame
firm_ret = firm_prices.pct_change().dropna()

    # Rename "Adj Close" to "TSLA"
firm_ret.rename(columns = {'Adj Close': 'Returns'}, inplace = True)

    # Convert index to monthly period date
firm_ret.index = firm_ret.index.to_period('M')
firm_ret.head(2)

[*********************100%%**********************]  1 of 1 completed


,Returns
Date,
2019-02,0.072776
2019-03,0.057250


In [6]:
# Merge the two datasets
data = firm_ret.join(ff3f)
data['const'] = 1
data.head(2)

,Returns,Mkt-RF,SMB,HML,RF,const
Date,,,,,,
2019-02,0.072776,0.034,0.0205,-0.0267,0.0018,1
2019-03,0.057250,0.011,-0.0305,-0.0417,0.0019,1


In [7]:
# Set up the data
    # Dependent variable (left side of the equal sign)
y = data['Returns'] - data['RF']
y.head(2)

Date
2019-02    0.070976
2019-03    0.055350
Freq: M, dtype: float64

In [8]:
    # Independent variable(s) (right side of the equal sign)
X = data[['const','Mkt-RF']]
X.head(2)

,const,Mkt-RF
Date,,
2019-02,1,0.034
2019-03,1,0.011


In [9]:
# Run single factor regression and store results in "res" object
res = sm.OLS(y,X).fit()
print(res.summary())
print("Reminder that 1 - Rsquared equals the percentage of the firms total risk that can be diversified away (idiosyncratic)")

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.528
Model:                            OLS   Adj. R-squared:                  0.520
Method:                 Least Squares   F-statistic:                     64.93
Date:                Mon, 08 Apr 2024   Prob (F-statistic):           4.89e-11
Time:                        11:55:07   Log-Likelihood:                 103.31
No. Observations:                  60   AIC:                            -202.6
Df Residuals:                      58   BIC:                            -198.4
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0149      0.006      2.581      0.0

In [10]:
#Adjusted Betas
l = 0.33
hBeta = res.params[1]
Adj_Beta = hBeta * (1-l) + l
print(f"The Adjusted Beta For {ticker} is: {Adj_Beta}")

The Adjusted Beta For MSFT is: 0.8944133010119615


In [11]:
tot_risk  = y.var()

print(f"The total risk of {ticker} equals: {tot_risk}")

indent()

beta = res.params['Mkt-RF']
sys_risk  = (beta**2) * data['Mkt-RF'].var() 

print(f"The systematic risk equals: {sys_risk}")

#Idiosyncratic Risk 
idio_risk = tot_risk - sys_risk
print(f"The idiosyncratic risk equals: {idio_risk}")

#As percentages
# Print as percentages of total risk
pct_sys_risk = sys_risk / tot_risk
pct_idio_risk = idio_risk / tot_risk
print(f'\n percent systematic risk = {pct_sys_risk: .4f} \n percent idiosyncratic risk = {pct_idio_risk: .4f}')

The total risk of MSFT equals: 0.00403226905849385


The systematic risk equals: 0.002129848072255152
The idiosyncratic risk equals: 0.001902420986238698

 percent systematic risk =  0.5282 
 percent idiosyncratic risk =  0.4718


In [12]:
#Single Factor Cost of Equity
Emrp = sum(ff3f['Mkt-RF']) / len(ff3f['Mkt-RF'])
Rf = 0.005 #Set this depending if RF is given to you in question, or if the question asks to use to most recent number
COE = (Rf + hBeta * Emrp) *12 * 100
COE_r = round(COE, 3)
print(f"The Annualized Cost of Equity for {ticker} is: {COE_r}%")

The Annualized Cost of Equity for MSFT is: 16.797%


In [13]:
#Single Factor P Val Interpretation
p_vals1 = res.pvalues
alpha = res.params[0]
if p_vals1['const'] < 0.01: #Set your significance level here
    print(f"Reject the null that true alpha is 0: we should reject the null hypothesis that the true alpha is 0")
    if alpha > 0:
        print(f"{ticker} is undervalued")
    else:
        print(f"{ticker} is overvalued")
else:
    print(f"Cannot reject the null that alpha is 0: we do not have sifficient eidence that the stock is mispriced")

Cannot reject the null that alpha is 0: we do not have sifficient eidence that the stock is mispriced


In [14]:
#Estimate the Fama-French three factor model using the data gathered above


# Set up X variables
X3 = data[['const','Mkt-RF','SMB','HML']]
# Run regression
res3 = sm.OLS(y,X3).fit()
print("Fama French 3 Factor Regression")
print(res3.summary())

Fama French 3 Factor Regression
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.737
Model:                            OLS   Adj. R-squared:                  0.722
Method:                 Least Squares   F-statistic:                     52.18
Date:                Mon, 08 Apr 2024   Prob (F-statistic):           3.16e-16
Time:                        11:55:07   Log-Likelihood:                 120.78
No. Observations:                  60   AIC:                            -233.6
Df Residuals:                      56   BIC:                            -225.2
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.012

In [15]:
#Three Factor Adjusted Beta
h3Beta = res3.params[1]
Adj_Beta = h3Beta * (1-l) + l
print(f"The Adjusted Three Factor Beta For {ticker} is: {Adj_Beta}")

The Adjusted Three Factor Beta For MSFT is: 0.9766412699638176


In [16]:
# Total risk of our stock (variance) in 3 factor (also works in single factor)

print(f"Again, the total risk of {ticker} equals: {tot_risk}")

indent()

# Systematic Risk
beta3 = res3.params['Mkt-RF']
sys_risk3  = (beta3**2) * data['Mkt-RF'].var() 

print(f"The 3 factor systematic risk equals: {sys_risk3}")

#Idiosyncratic Risk 
idio_risk3 = tot_risk - sys_risk
print(f"The 3 factor idiosyncratic risk equals: {idio_risk3}")

Again, the total risk of MSFT equals: 0.00403226905849385


The 3 factor systematic risk equals: 0.0027956383084422405
The 3 factor idiosyncratic risk equals: 0.001902420986238698


In [17]:
# Print as percentages of total risk
pct_sys_risk = sys_risk3 / tot_risk
pct_idio_risk = idio_risk3 / tot_risk
print(f'\n 3 factor percent systematic risk = {pct_sys_risk: .4f} \n 3 factor percent idiosyncratic risk = {pct_idio_risk: .4f}')


 3 factor percent systematic risk =  0.6933 
 3 factor percent idiosyncratic risk =  0.4718


In [18]:
#Pvalue and Alpha for 3 factor model
p_vals = res3.pvalues
if p_vals['const'] < 0.01: #Set your significance level here
    print(f"Reject the null that true alpha is 0: we should reject the null hypothesis that the true alpha is 0")
else:
    print(f"Cannot reject the null that alpha is 0: we do not have sifficient eidence that the stock is mispriced")

Reject the null that true alpha is 0: we should reject the null hypothesis that the true alpha is 0


In [19]:
#Cost of Equity in 3 factor
#We already have our risk free rate variabe "Rf" 
#and the expected Market RIsk Premium "Ermp" and beta for the stock "hBeta"
#We now need the exp ret for our other two factors and their betas
#also we need to update the beta "hBeta" to "h3Beta"

#Expected Returns
ErSMB = sum(ff3f['SMB']) / len(ff3f['SMB'])
ErHML = sum(ff3f['HML']) / len(ff3f['HML'])

#SMB, HML, and 3h Betas
Bhml = res3.params[3]
Bsmb = res3.params[2]
h3Beta = res3.params[1]
print(f" HML - {Bhml} and SMB - {Bsmb} and the market beta {h3Beta}")


f3COE = Rf + (h3Beta * Emrp) + (Bsmb * ErSMB) + (Bhml * ErHML)
Af3COE = (f3COE * 12) * 100


print(f"The **Annualized*** Cost of Equity for {ticker} is: {Af3COE.round(3)}%")


 HML - -0.4930902533532801 and SMB - -0.5729045763231198 and the market beta 0.9651362238265935
The **Annualized*** Cost of Equity for MSFT is: 20.416%
